In [10]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

from os.path import join
import xarray as xr
import numpy as np
import pandas as pd
import admix

import sys

sys.path.append(
    "/u/project/pasaniuc/pasaniucdata/admixture/projects/admix-prs-uncertainty"
)
import common

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [10]:
from os.path import join
import xarray as xr
import numpy as np
import pandas as pd
import admix
import matplotlib.pyplot as plt

PLINK_DIR = "/u/project/pasaniuc/pasaniucdata/admixture/projects/admix-prs-uncertainty/data/PLINK/"

dset_eur_train = admix.io.read_plink(join(PLINK_DIR, "eur_train/chr22"))
dset_eur_val = admix.io.read_plink(join(PLINK_DIR, "eur_val/chr22"))
dset_eur_test = admix.io.read_plink(join(PLINK_DIR, "eur_test/chr22"))

dset_admix = admix.io.read_plink(join(PLINK_DIR, "admix/chr22"))
dset = xr.concat([dset_eur_train, dset_eur_val, dset_eur_test, dset_admix], dim="indiv")

np.random.seed(1234)
sim = admix.simulate.continuous_pheno_1pop(
    dset.isel(snp=np.arange(1000)), var_g=0.001, var_e=1.0, n_sim=10
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
_geno_mult_mat: 100%|██████████| 1/1 [00:15<00:00, 15.42s/it]


In [12]:
beta = sim["beta"].copy()
pheno = sim["pheno"].copy()
pheno.index = pheno.index.str.split("_", expand=True)
pheno.index.names = ["FID", "IID"]
pheno = pheno.reset_index()

admix.tools.plink_assoc(
    join(PLINK_DIR, "eur_train/chr22"), pheno, out_prefix="out/plink"
)

